<a href="https://colab.research.google.com/github/agatagruza/private-ai/blob/master/SPAIC_Project10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 10: Simple Linear Model

Create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). In addition, you must do so with both the data and the model being located on Bob's machine.

In [0]:
pip install syft

In [2]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)

W0724 08:56:07.303856 139818918893440 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0724 08:56:07.327231 139818918893440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
bob = sy.VirtualWorker(hook, id = "bob")

In [0]:
#Initializing data that are as small as possible
my_input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
#We want to learn a real linear relationship between column input and target 
my_target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [6]:
my_input #input is on Bob machine 

(Wrapper)>[PointerTensor | me:98050856396 -> bob:20492789624]

In [7]:
my_target #target is on Bob machine 

(Wrapper)>[PointerTensor | me:35863372609 -> bob:40906876527]

In [0]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)
# So now Bob has input data, target data, and set of weights

In [18]:
# FORWARD PROPAGATION
for i in range(10):
  pred = my_input.mm(weights) # prediction is a pointer 
  loss = ((pred - my_target)**2).sum() #MEAN SQARE ERROR LOSS 
  loss.backward() #BACKPROPAGATIO 

  # weight update
  weights.data.sub_(weights.grad * 0.1) #alpha=0.1 
  weights.grad *= 0  #zero out our gradient
  print(loss.get().data)   

tensor(1.0215e-10)
tensor(6.5379e-11)
tensor(4.1952e-11)
tensor(2.7028e-11)
tensor(1.7156e-11)
tensor(1.0924e-11)
tensor(6.9472e-12)
tensor(4.4816e-12)
tensor(2.8682e-12)
tensor(1.8357e-12)


***Once we have moved our data and our weights to row machine, block of code [18] just using native normal run-of-the-mill PyTorch.***

#Garbage Collection and Common Errors

In PySyft , there is a provision for garbage collection. 
 
***PySyft always assumes that when you created that tensor and send it to someone, you should continue to control the life cycle of that tensor.*** That means, when you delete a pointer to the tensor you have created and sent, the tensor that you're pointing to should also get deleted. 

In [19]:
bob

<VirtualWorker id:bob #objects:4>

In [0]:
# Clear all the objects in bob
bob = bob.clear_objects()

In [22]:
#bob_object is now empty
bob._objects

{}

In [23]:
# creating tensor and pointing to bob, x here is a pointer
x = th.tensor([1,2,3,4,5]).send(bob)
bob._objects # we can see that a bob object is here

{46308565627: tensor([1, 2, 3, 4, 5])}

In [0]:
del x 

In [25]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [35]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#By default GC is set to True.
#When OFF, GC will NOT send a message to bob saying
# bob should delete tensor when they get removed. 
x.child.garbage_collect_data

True

In [36]:
bob._objects

{39280460036: tensor([1, 2, 3, 4, 5]), 69136289033: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [41]:
bob._objects

{39280460036: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

Jupyter Notebook has ***Out*** command, where every command you have executed is stored. Occxasionally, those commands will be cashced in a way that GC will no longer collect those objects. Pointer will never get GC. Even if you remobve x, pointer still lives on. It is not because some issues with PySyft. It's just an known issue, sort of UNKNOWN GOTCHA when you use Jupyet Notebook. 

In [0]:
bob = bob.clear_objects()

In [46]:
bob._objects

{}

In [0]:
#Sending tensor to Bob 1000 times
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

***bob._object should still have only one. Reason behind that that for every loop iteration, loop reasssign x and delete the tensor that was returned. And that caused it to be GC from the remote machine. It's important because it means when you're doing federated learning and iterate through the loop, you are getting a new prediction tensor, new loss tensor, and the intermediate tensors that go into executing each of these parts. If we end up deleting our referense to that execution,  then remote machine will also remove it.***

In [50]:
bob._objects

{79782842950: tensor([1, 2, 3, 4, 5])}

##WELL KNOWN ERRORS

**COMMON ERROR 1:**

In [0]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob) # x is pointer to bob
y = th.tensor([1, 2, 3, 4, 5]) # y is a dataset tensor

In [52]:
z = x + y

TensorsNotCollocatedException: ignored

***PureTorchTensorFoundError - expected error. It mans one of the tensors is a regular tensor, and the other tensor is a POINTER to the tensor***
Bottom of the error message explains everything:  **"You tried to call a method involving two tensors where one tensor is actually located on another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor."**

**COMMON ERROR 2:**

In [0]:
alice = sy.VirtualWorker(hook, id = "alice")
x = th.tensor([1, 2, 3, 4, 5]).send(bob)
y = th.tensor([1, 2, 3, 4, 5]).send(alice)

In [55]:
z = x + y

TensorsNotCollocatedException: ignored

**"You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:2> while the other is on <VirtualWorker id:alice #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine."** 

In any of the above cases, just follow the recommended instructions an dyou will led to the right series of operations that you're interesting in performing in the first place. 